### Introduction

The objective of this example is to run the basic calibration steps on raw frames.
By the end of this example, we will know how to:

- Pre-process raw frames to remove the overscan section and extract the science section
- Remove cosmic rays from individual frames using the L.A. algorithm (Van Dokkum+2005)
- Produce calibrated bias, dark and pixelflat frames

Let's start with some imports and function definitions

In [1]:
import os

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import GAINS, RDNOISES
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata

from lvmdrp.functions import imageMethod

### Downloading 2D simulations

Before we begin, we need to:
- Define paths to input and output data
- Download some data to run through the pipeline

In this case, we'll download 2D simulated data.

In [2]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

# now we download the example data
fetch_example_data(url=LVM_UNAM_URL, name="sim_2d_old", dest_path=data_path)

[INFO]: downloading example data to /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old.zip
[INFO]:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
[INFO]:                                  Dload  Upload   Total   Spent    Left  Speed
[INFO]: 
[INFO]:   0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
[INFO]:   0 2149M    0 4916k    0     0  16.8M      0  0:02:07 --:--:--  0:02:07 16.7M
[INFO]:   2 2149M    2 44.2M    0     0  34.6M      0  0:01:02  0:00:01  0:01:01 34.6M
[INFO]:   3 2149M    3 83.4M    0     0  36.6M      0  0:00:58  0:00:02  0:00:56 36.6M
[INFO]:   5 2149M    5  122M    0     0  37.4M      0  0:00:57  0:00:03  0:00:54 37.4M
[INFO]:   7 2149M    7  162M    0     0  37.8M      0  0:00:56  0:00:04  0:00:52 37.8M
[INFO]:   9 2149M    9  201M    0     0  38.0M      0  0:00:56  0:00:05  0:00:51 39.3M
[INFO]:  11 2149M   11  240M    0     0  38.2M      0  0:00:56  0:00:06  0:00:50 39.1M
[INFO]:  13 214

In order to organize our simulated data and target specific type of frames better,
we extract relevant metadata from the frames and group them by image type, camera
and exposure time.

In [3]:
# extract metadata
frames_table = get_frames_metadata(path=data_path)
# group by imagetyp, camera and exptime
frames_table = frames_table.group_by(["imagetyp", "camera", "exptime"])
# define groups and unique categories in those groups
frames_groups = frames_table.groups
unique_groups = frames_groups.keys

[INFO]: extracting metadata from 252 frames


100%|######################################################################################################| 252/252 [00:51<00:00,  4.90it/s]

[INFO]: successfully extracted metadata


We will start by processing calibration frames: bias, dark and pixelflats

Let's start with the biases for the blue channel of spectrograph #1...

In [4]:
gains_b = ",".join(map(str, GAINS["b"]))
rdnoises_b = ",".join(map(str, RDNOISES["b"]))

In [5]:
# OK, biases have exptime != 0 for some reason...
biases = frames_groups[(unique_groups["imagetyp"]=="bias")&(unique_groups["camera"]=="b1")]
biases

imagetyp,spec,camera,expnum,exptime,path
str9,str3,str2,str8,float64,str98
bias,sp1,b1,00000018,900.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/bias/sdR-b1-00000018.fit.gz
bias,sp1,b1,00000017,900.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/bias/sdR-b1-00000017.fit.gz
bias,sp1,b1,00000001,900.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/bias/sdR-b1-00000001.fit.gz
bias,sp1,b1,00000002,900.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/bias/sdR-b1-00000002.fit.gz
bias,sp1,b1,00000000,900.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/bias/sdR-b1-00000000.fit.gz
bias,sp1,b1,00000003,900.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/bias/sdR-b1-00000003.fit.gz
bias,sp1,b1,00000019,900.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/bias/sdR-b1-00000019.fit.gz
bias,sp1,b1,00000004,900.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/bias/sdR-b1-00000004.fit.gz


In [6]:
# let's pre-process the first bias frame
bias_frame = biases[0]
bias_path = bias_frame["path"]
bias_camera, bias_expnum = bias_frame["camera"], bias_frame["expnum"]
imageMethod.preprocRawFrame_drp(
    in_image=bias_path,
    out_image=os.path.join(output_path, f"lvm-pbias-{bias_camera}-{bias_expnum}.fits"),
    subtract_overscan=False, unit="e-", compute_error=True,
    assume_imagetyp="bias", assume_gain=gains_b, assume_rdnoise=rdnoises_b
)

In [7]:
darks = frames_groups[(unique_groups["imagetyp"]=="dark")&(unique_groups["camera"]=="b1")]
darks

imagetyp,spec,camera,expnum,exptime,path
str9,str3,str2,str8,float64,str98
dark,sp1,b1,00000024,10.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/dark/sdR-b1-00000024.fit.gz
dark,sp1,b1,00000025,100.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/dark/sdR-b1-00000025.fit.gz
dark,sp1,b1,00000026,200.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/dark/sdR-b1-00000026.fit.gz
dark,sp1,b1,00000027,300.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/dark/sdR-b1-00000027.fit.gz
dark,sp1,b1,00000028,400.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/dark/sdR-b1-00000028.fit.gz
dark,sp1,b1,00000029,500.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/dark/sdR-b1-00000029.fit.gz
dark,sp1,b1,00000030,750.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/dark/sdR-b1-00000030.fit.gz
dark,sp1,b1,00000031,1000.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/dark/sdR-b1-00000031.fit.gz
dark,sp1,b1,00000032,1500.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/dark/sdR-b1-00000032.fit.gz


In [8]:
# now let's pre-process one dark
dark_frame = darks[1]
dark_path = dark_frame["path"]
dark_camera, dark_expnum = dark_frame["camera"], dark_frame["expnum"]
imageMethod.preprocRawFrame_drp(
    in_image=dark_path,
    out_image=os.path.join(output_path, f"lvm-pdark-{dark_camera}-{dark_expnum}.fits"),
    subtract_overscan=False, unit="e-", compute_error=True,
    assume_imagetyp="dark", assume_gain=gains_b, assume_rdnoise=rdnoises_b
)

In [9]:
pixelflats = frames_groups[(unique_groups["imagetyp"]=="dark")&(unique_groups["camera"]=="b1")]
pixelflats

pixelflat_frame = pixelflats[4]
pixelflat_path = pixelflat_frame["path"]
pixelflat_camera, pixelflat_expnum = pixelflat_frame["camera"], pixelflat_frame["expnum"]
imageMethod.preprocRawFrame_drp(
    in_image=pixelflat_path,
    out_image=os.path.join(output_path, f"lvm-ppixelflat-{pixelflat_camera}-{pixelflat_expnum}.fits"),
    subtract_overscan=False, unit="e-", compute_error=True,
    assume_imagetyp="flatfield", assume_gain=gains_b, assume_rdnoise=rdnoises_b
)

In [10]:
# because we will be using individual exposures to calibrate, let's reject those cosmic rays
imageMethod.LACosmic_drp(
    in_image=os.path.join(output_path, f"lvm-pbias-{bias_camera}-{bias_expnum}.fits"),
    out_image=os.path.join(output_path, f"lvm-rbias-{bias_camera}-{bias_expnum}.fits"),
    iter=3, increase_radius=1, flim=1.3, parallel="auto"
)
imageMethod.LACosmic_drp(
    in_image=os.path.join(output_path, f"lvm-pdark-{dark_camera}-{dark_expnum}.fits"),
    out_image=os.path.join(output_path, f"lvm-rdark-{dark_camera}-{dark_expnum}.fits"),
    iter=3, increase_radius=1, flim=1.3, parallel="auto"
)
imageMethod.LACosmic_drp(
    in_image=os.path.join(output_path, f"lvm-ppixelflat-{pixelflat_camera}-{pixelflat_expnum}.fits"),
    out_image=os.path.join(output_path, f"lvm-rpixelflat-{pixelflat_camera}-{pixelflat_expnum}.fits"),
    iter=3, increase_radius=1, flim=1.3, parallel="auto"
)

In [11]:
# the next step will be to reduce each calibration frame
# in general this implies:
#   * subtracting the bias
#   * subtracting the dark
#   * and flat fielding

# start with the dark frame
imageMethod.basicCalibration_drp(
    in_image=os.path.join(output_path, f"lvm-rdark-{dark_camera}-{dark_expnum}.fits"),
    out_image=os.path.join(output_path, f"lvm-cdark-{dark_camera}-{dark_expnum}.fits"),
    in_bias=os.path.join(output_path, f"lvm-rbias-{bias_camera}-{bias_expnum}.fits")
)
# and continue with the pixelflat
imageMethod.basicCalibration_drp(
    in_image=os.path.join(output_path, f"lvm-rpixelflat-{pixelflat_camera}-{pixelflat_expnum}.fits"),
    out_image=os.path.join(output_path, f"lvm-cpixelflat-{pixelflat_camera}-{pixelflat_expnum}.fits"),
    in_bias=os.path.join(output_path, f"lvm-rbias-{bias_camera}-{bias_expnum}.fits"),
    in_dark=os.path.join(output_path, f"lvm-cdark-{dark_camera}-{dark_expnum}.fits")
)

In [12]:
fiberflats = frames_groups[(unique_groups["imagetyp"]=="fiberflat")&(unique_groups["camera"]=="b1")]
fiberflats

imagetyp,spec,camera,expnum,exptime,path
str9,str3,str2,str8,float64,str98
fiberflat,sp1,b1,00000022,25.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/fiberflat/sdR-b1-00000022.fit.gz
fiberflat,sp1,b1,00000005,25.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/fiberflat/sdR-b1-00000005.fit.gz
fiberflat,sp1,b1,00000023,25.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/fiberflat/sdR-b1-00000023.fit.gz
fiberflat,sp1,b1,00000006,25.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/fiberflat/sdR-b1-00000006.fit.gz


In [13]:
# Now that we have calibration frames, let's calibrate a fiberflat and an arc

fiberflat_frame = fiberflats[0]
fiberflat_path = fiberflat_frame["path"]
fiberflat_camera, fiberflat_expnum = fiberflat_frame["camera"], fiberflat_frame["expnum"]
imageMethod.preprocRawFrame_drp(
    in_image=fiberflat_path,
    out_image=os.path.join(output_path, f"lvm-pfiberflat-{fiberflat_camera}-{fiberflat_expnum}.fits"),
    subtract_overscan=False, unit="e-", compute_error=True,
    assume_imagetyp="fiberflat", assume_gain=gains_b, assume_rdnoise=rdnoises_b
)
imageMethod.LACosmic_drp(
    in_image=os.path.join(output_path, f"lvm-pfiberflat-{fiberflat_camera}-{fiberflat_expnum}.fits"),
    out_image=os.path.join(output_path, f"lvm-rfiberflat-{fiberflat_camera}-{fiberflat_expnum}.fits"),
    iter=3, increase_radius=1, flim=1.3, parallel="auto"
)
imageMethod.basicCalibration_drp(
    in_image=os.path.join(output_path, f"lvm-rfiberflat-{fiberflat_camera}-{fiberflat_expnum}.fits"),
    out_image=os.path.join(output_path, f"lvm-cfiberflat-{fiberflat_camera}-{fiberflat_expnum}.fits"),
    in_bias=os.path.join(output_path, f"lvm-rbias-{bias_camera}-{bias_expnum}.fits"),
    in_dark=os.path.join(output_path, f"lvm-cdark-{dark_camera}-{dark_expnum}.fits"),
    in_pixelflat=os.path.join(output_path, f"lvm-cpixelflat-{pixelflat_camera}-{pixelflat_expnum}.fits")
)

In [14]:
arcs = frames_groups[(unique_groups["imagetyp"]=="arc")&(unique_groups["camera"]=="b1")]
arcs

imagetyp,spec,camera,expnum,exptime,path
str9,str3,str2,str8,float64,str98
arc,sp1,b1,00000021,4.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/arc/sdR-b1-00000021.fit.gz
arc,sp1,b1,00000007,4.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/arc/sdR-b1-00000007.fit.gz
arc,sp1,b1,00000020,4.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/arc/sdR-b1-00000020.fit.gz
arc,sp1,b1,00000008,4.14,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d_old/arc/sdR-b1-00000008.fit.gz


In [15]:
arc_frame = arcs[0]
arc_path = arc_frame["path"]
arc_camera, arc_expnum = arc_frame["camera"], arc_frame["expnum"]
imageMethod.preprocRawFrame_drp(
    in_image=arc_path,
    out_image=os.path.join(output_path, f"lvm-parc-{arc_camera}-{arc_expnum}.fits"),
    subtract_overscan=False, unit="e-", compute_error=True,
    assume_imagetyp="arc", assume_gain=gains_b, assume_rdnoise=rdnoises_b
)
imageMethod.LACosmic_drp(
    in_image=os.path.join(output_path, f"lvm-parc-{arc_camera}-{arc_expnum}.fits"),
    out_image=os.path.join(output_path, f"lvm-rarc-{arc_camera}-{arc_expnum}.fits"),
    iter=3, increase_radius=1, flim=1.3, parallel="auto"
)
imageMethod.basicCalibration_drp(
    in_image=os.path.join(output_path, f"lvm-rarc-{arc_camera}-{arc_expnum}.fits"),
    out_image=os.path.join(output_path, f"lvm-carc-{arc_camera}-{arc_expnum}.fits"),
    in_bias=os.path.join(output_path, f"lvm-rbias-{bias_camera}-{bias_expnum}.fits"),
    in_dark=os.path.join(output_path, f"lvm-cdark-{dark_camera}-{dark_expnum}.fits"),
    in_pixelflat=os.path.join(output_path, f"lvm-cpixelflat-{pixelflat_camera}-{pixelflat_expnum}.fits")
)